In [33]:
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import Subset

In [2]:
dataset = load_dataset('knowledgator/events_classification_biotech')

/home/kodama/.myenv/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for knowledgator/events_classification_biotech contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/knowledgator/events_classification_biotech
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
classes = set(i[0] for i in dataset['train']["all_labels"])
class2id = {class_: id for id, class_ in enumerate(classes)}
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [4]:
def preprocess_function(example):
    text = f"{example['title']}.\n{example['content']}"
    labels = [0.0 for _ in range(len(classes))]
    label_id = class2id[example["all_labels"][0]]
    labels[label_id] = 1.0

    example = tokenizer(text, truncation=True, max_length=512, padding="max_length")
    example["labels"] = labels
    return example

In [5]:
tokenized_dataset = dataset.map(preprocess_function)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

Map:   0%|          | 0/2759 [00:00<?, ? examples/s]

Map:   0%|          | 0/381 [00:00<?, ? examples/s]

In [6]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'content', 'target organization', 'all_labels', 'all_labels_concat', 'label 1', 'label 2', 'label 3', 'label 4', 'label 5', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2759
    })
    test: Dataset({
        features: ['title', 'content', 'target organization', 'all_labels', 'all_labels_concat', 'label 1', 'label 2', 'label 3', 'label 4', 'label 5', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 381
    })
})

In [14]:
n_labels = tokenized_dataset['train']['labels'].sum(dim=0)

In [20]:
def top_n_indices(n_labels, n):
    top_indices = []
    for _ in range(n):
        i = n_labels.argmax().item()
        n_labels[i] = 0.
        top_indices.append(i)
    return top_indices


In [24]:
top_indices = top_n_indices(n_labels, 2)

In [39]:
boolian = [i for i, _ in enumerate(tokenized_dataset['train']['labels'].argmax(dim=1)) if _ in top_indices]

In [41]:
subset = Subset(tokenized_dataset['train'], indices=boolian)

In [42]:
len(subset)

394